In [1]:
import os
import json

# import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization, Input, Concatenate, AvgPool2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.regularizers import l1
from sklearn.preprocessing import MultiLabelBinarizer
from cnn_model import ImageClassificationModel

# import miscellaneous modules
import matplotlib.pyplot as plt
from imutils import paths
from six.moves import cPickle as pickle
from PIL import Image
import cv2
import os
from tqdm import tqdm
import numpy as np
import shutil
import tarfile
from multiprocessing import Manager, Process, Pool

Using TensorFlow backend.


In [2]:
with open('train_annotations.json', 'r') as file:
    train_annotations = json.load(file)

with open('validation_annotations.json', 'r') as file:
    validation_annotations = json.load(file)

In [3]:
target_size = (224,224)
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = os.listdir('train')
nb_validation_samples = os.listdir('validation')
epochs = 50
batch_size = 32

In [4]:
imagePaths = list(paths.list_images('train'))
shape = (len(imagePaths),)+target_size+(3,)
train_x = np.zeros(shape=shape, dtype=np.float16)
train_y = []
for i in tqdm(range(len(imagePaths))):
    imagePath = imagePaths[i]
    image = Image.open(imagePath)
    image = image.resize(size=target_size, resample=Image.LANCZOS)
    image = np.array(image)
    if (len(image.shape) == 2):
        image = np.repeat(image[:, :, np.newaxis], 3, axis=2)
    # print(image.shape)
    train_x[i]=image
    labels = set()
    for category in train_annotations[imagePath.split(os.path.sep)[-1]]['categories']: labels.add(category)
    train_y.append(list(labels))

100%|██████████| 49669/49669 [20:55<00:00, 39.55it/s]


In [13]:
print(train_x.shape)
train_y = np.array(train_y)

In [19]:
imagePaths = list(paths.list_images('validation'))
shape = (len(imagePaths),)+target_size+(3,)
validation_x = np.zeros(shape=shape, dtype=np.float16)
validation_y = []
for i in tqdm(range(len(imagePaths))):
    imagePath = imagePaths[i]
    image = Image.open(imagePath)
    image = image.resize(size=target_size, resample=Image.LANCZOS)
    image = np.array(image)
    if (len(image.shape) == 2):
        image = np.repeat(image[:, :, np.newaxis], 3, axis=2)
    # print(image.shape)
    validation_x[i]=image
    labels = set()
    for category in validation_annotations[imagePaths[i].split(os.path.sep)[-1]]['categories']: labels.add(category)
    validation_y.append(list(labels))

0%|          | 0/100 [00:00<?, ?it/s]


KeyError: 'COCO_train2014_000000339579.jpg'

In [7]:
print(validation_x.shape)
validation_y = np.array(validation_y)

NameError: name 'train_data' is not defined

In [0]:
with open('instances_train2014.json','r') as file:
    instances = json.load(file)

labels = np.array([category['name'] for category in instances['categories']])

In [0]:
train_datagen = ImageDataGenerator(
        rotation_range=25, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1./255,
        shear_range=0.2,
        fill_mode='nearest',
        zoom_range=0.2,
        horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

model = ImageClassificationModel.build(target_size[0], target_size[1], len(labels), 'softmax')
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.95), metrics=['accuracy'])

In [0]:
stop_early = EarlyStopping(monitor='val_loss', patience=3)
reduceLR = ReduceLROnPlateau(monitor='val_loss', paitence = 1, factor=0.2, min_lr = 0.0001)
callbacks = [stop_early, reduceLR]
model.fit_generator(
    train_datagen.flow(train_x, train_y, batch_size= batch_size, target_size=target_size),
    steps_per_epoch = nb_train_samples//batch_size,
    validation_data=validation_datagen.flow(validation_x, validation_y, batch_size=batch_size, target_size=target_size),
    validation_steps = nb_validation_samples//batch_size,
    callbacks = callbacks)

In [0]:
img = Image.open('test/COCO_train2014_000000000109.jpg') # image extension *.png,*.jpg
img = img.resize((224,224), Image.ANTIALIAS)
arr = np.array(img)
arr=np.expand_dims(arr,0)
arr=arr/255
total_model.predict(arr)